In [ ]:
!(@isdefined cleanup_hook) && IJulia.push_postexecute_hook(() -> (empty!(In); empty!(Out); GC.gc(true))); cleanup_hook = true;

In [ ]:
using ProgressMeter, PyCall, PyPlot, Images, HDF5,NaNStatistics, Statistics, DSP, Lasso, JLD2
using _Data, _Math

In [ ]:
@pyimport numpy
@pyimport sklearn.decomposition as decomposition
@pyimport scipy.stats as stats
@pyimport matplotlib.colors as mpl_colors
@pyimport matplotlib.cm as cm 
@pyimport sklearn.cluster as cluster

In [ ]:
include("../../../functions/func_map.jl")
include("../../../functions/func_stat.jl")
include("../../../functions/func_data.jl")
include("../../../functions/func_plot.jl")

In [ ]:
data_info_dict = load("/home/chuyu/Notebooks/project_place_cell/figures/chuyu/figure_data_info.jld2")

data_info_all = []
for key in keys(data_info_dict)
    append!(data_info_all, data_info_dict[key])
end

In [ ]:
analyzer = "chuyu"

# Check data

In [ ]:
function map_dots_from_boundary(chosen_point; x, y, k)
    distance_from_chosen = distance_from(numpy.array(x), chosen_point)
    weights = 1 ./(distance_from_chosen .^(k))
    
    mapped_point = sum(weights .* y)/sum(weights)
    return mapped_point
end

function boundary_morphing_map(example_map, mask_pixels, mask_pixels_array_map, bin_interval)
    mapped_map_sum = zeros(Float32, size(example_map))
    mapped_map_count = zeros(Float32, size(example_map))
    for i in 1:length(mask_pixels_array_map)
        index = mask_pixels_array_map[i]./bin_interval
        value = example_map[mask_pixels[i]]
        mapped_map_sum[floor(Int32, index[1]-1):ceil(Int32, index[1]+1), floor(Int32, index[2]-1):ceil(Int32, index[2]+1)] .+= value
        mapped_map_count[floor(Int32, index[1]-1):ceil(Int32, index[1]+1), floor(Int32, index[2]-1):ceil(Int32, index[2]+1)] .+= 1
    end

    mapped_map = mapped_map_sum./mapped_map_count;
    return mapped_map
end

In [ ]:
for which_data = 1:length(data_info_all)
data_info = data_info_all[which_data]

experiment_filename_1 = data_info[3]
server_1 = data_info[2]

experiment_filename_2 = data_info[1]
server_2 = data_info[4]

experimenter = data_info[5]
ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")

    
    
file_folder_1 = joinpath(data_path(ds_save_cy_2), "place_cell_windows")
all_files = readdir(file_folder_1)
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_2, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
println(candidate_filename[which_file])


end

In [ ]:
for which_data in 1:length(data_info_all)
    try
    data_info = data_info_all[which_data]

    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]

    experiment_filename_2 = data_info[3]
    server_2 = data_info[4]

    experimenter = data_info[5]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

    ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
    ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")
    ds_save_analyzer_2 = Dataset(experiment_filename_2, analyzer, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")



    
    # println(which_data)
    
    path_target = data_path(ds_save_cy_1)
    run(`ls -lha $path_target/boundary_morph_earlylate.h5`)

    catch e
        println(e)
    end
    

end

# Best match

In [ ]:
for which_data = 1:length(data_info_all)
data_info = data_info_all[which_data]

experiment_filename_1 = data_info[1]
server_1 = data_info[2]

experiment_filename_2 = data_info[1]
server_2 = data_info[2]

experimenter = data_info[5]
ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")

    
# orientation-corrected background image and chamber roi image
orientation_correction_file = h5open(joinpath(data_path(ds_save_cy_1), "chamber_geometry_$(experiment_filename_1).h5"))
chamber_roi_1 = read(orientation_correction_file,"chamber_roi")
countour_array_1 = read(orientation_correction_file,"countour_array")
close(orientation_correction_file)

    
img_bg_1 = h5open(ds_save_1, "behavior.h5"; raw = true) do file
    read(file, "img_bg")
end;

img_bg_end_1 = img_bg_1[:,:,end]
w = size(img_bg_end_1, 1)
l = size(img_bg_end_1, 2)
# check how well they match
img_bg_end_1[chamber_roi_1.!=0].=NaN
    
# orientation-corrected background image and chamber roi image
orientation_correction_file = h5open(joinpath(data_path(ds_save_cy_2), "chamber_geometry_$(experiment_filename_2).h5"))
chamber_roi_2 = read(orientation_correction_file,"chamber_roi")
countour_array_2 = read(orientation_correction_file,"countour_array")
close(orientation_correction_file)
    
img_bg_2 = h5open(ds_save_2, "behavior.h5"; raw = true) do file
    read(file, "img_bg")
end;

img_bg_end_2 = img_bg_2[:,:,end]
w = size(img_bg_end_2, 1)
l = size(img_bg_end_2, 2)

# check how well they match
img_bg_end_2[chamber_roi_2.!=0].=NaN
nr_dots = maximum([size(countour_array_1,1), size(countour_array_2,1)])
    
offset = 0

nr_roll_1 = 0
nr_roll_2 = offset

x1 = numpy.interp(numpy.linspace(0, 1, nr_dots), numpy.linspace(0, 1, size(countour_array_1, 1)), countour_array_1[:,1])
x2 = numpy.interp(numpy.linspace(0, 1, nr_dots), numpy.linspace(0, 1, size(countour_array_1, 1)), countour_array_1[:,2])

y1 = numpy.interp(numpy.linspace(0, 1, nr_dots), numpy.linspace(0, 1, size(countour_array_2, 1)), countour_array_2[:,1])
y2 = numpy.interp(numpy.linspace(0, 1, nr_dots), numpy.linspace(0, 1, size(countour_array_2, 1)), countour_array_2[:,2]);

x1 = numpy.roll(x1, nr_roll_1)
x2 = numpy.roll(x2, nr_roll_1);
y1 = numpy.roll(y1, nr_roll_2)
y2 = numpy.roll(y2, nr_roll_2);

x = [[x1[i], x2[i]] for i in 1:nr_dots]
y = [[y1[i], y2[i]] for i in 1:nr_dots];


# figure()
# scatter(first.(x), last.(x), c=1:length(x), s =3)
# imshow(img_bg_end_1', cmap="gray", vmax=300)
# title(experiment_filename_1)

# figure()
# scatter(first.(y), last.(y), c=1:length(y), s =3)
# imshow(img_bg_end_2', cmap="gray", vmax=300)
# title(experiment_filename_2)
    
chamber_dots = findall(chamber_roi_1.==1)
chamber_dots_array = [[chamber_dots[i][1], chamber_dots[i][2]] for i in 1:length(chamber_dots)];

chamber_dots_array_sample = chamber_dots_array[numpy.random.choice(1:length(chamber_dots_array), 1000, replace=false)]
chamber_dots_array_reconstruct = map_dots_from_boundary.(chamber_dots_array_sample, x= x, y =x, k= 2);
chamber_dots_array_map = map_dots_from_boundary.(chamber_dots_array_sample, x= x, y =y, k= 2);
    
err = norm.(chamber_dots_array_sample .- chamber_dots_array_reconstruct)
# figure()
# hist(err)
    
    
file_folder_1 = joinpath(data_path(ds_save_cy_2), "place_cell_windows")
all_files = readdir(file_folder_1)
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_2, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
@assert(length(candidate_filename[which_file]) == 2)
save_file_name = candidate_filename[which_file][2]
info_filename = joinpath(file_folder_1, save_file_name)
file = h5open(info_filename, "r")
place_map_all_1 = HDF5.readmmap(file["place_map_all"])
close(file)


save_file_name = candidate_filename[which_file][1]
info_filename = joinpath(file_folder_1, save_file_name)
file = h5open(info_filename, "r")

place_map_all_2 = HDF5.readmmap(file["place_map_all"])
close(file)
    
    
compare_map_file = h5open(joinpath(data_path(ds_save_cy_2), "compare_map_results_original.h5"))
tel_place_cell_index = read(compare_map_file,"tel_place_cell_index")
close(compare_map_file)    
    
precise_loc = false
save_file_name = "compare_map_results_early_late.h5"
compare_filename = joinpath(data_path(ds_save_cy_2), save_file_name)
file = h5open(compare_filename, "r")
peak_loc_map_1 = HDF5.readmmap(file["peak_loc_map_1_early"])
peak_loc_map_2 = HDF5.readmmap(file["peak_loc_map_1_late"])
if haskey(file, "peak_loc_map_1_precise")
    precise_loc = true
    peak_loc_map_1_precise  = HDF5.readmmap(file["peak_loc_map_1_precise"])
    peak_loc_map_2_precise  = HDF5.readmmap(file["peak_loc_map_2_precise"])
end
close(file)
    
    
chosen_neuron = tel_place_cell_index;
which_neuron = chosen_neuron[1]
    
n_pos = size(place_map_all_1,1);
for_place_calculation_file = h5open(joinpath(data_path(ds_save_cy_1), "for_place_calculation_chamber_geometry_$(experiment_filename_1)_n$(n_pos).h5"))
x_bins = read(for_place_calculation_file,"x_bins")
y_bins = read(for_place_calculation_file,"y_bins")
mask_valid = read(for_place_calculation_file,"mask_valid")
close(for_place_calculation_file)
    
bin_interval = x_bins[2] - x_bins[1]
    
mask_pixels = findall(mask_valid)
mask_pixels_array = [[(mask_pixels[i][1] .-0.5)*bin_interval, (mask_pixels[i][2] .-0.5)*bin_interval] for i in 1:length(mask_pixels)]
mask_pixels_array_map = map_dots_from_boundary.(mask_pixels_array, x= x, y =y, k= 2);

figure()
scatter(first.(mask_pixels_array), last.(mask_pixels_array), c=1:length(mask_pixels_array), cmap="Dark2")
scatter(first.(x), last.(x), c=1:length(x))
# axis("square")
title(experiment_filename_1)
figure()
scatter(first.(mask_pixels_array_map), last.(mask_pixels_array_map), c=1:length(mask_pixels_array_map), cmap="Dark2")
scatter(first.(y), last.(y), c=1:length(y))
axis("square")
title(experiment_filename_2)
    
    
    


mapped_map_all = fill(NaN32, size(place_map_all_1))
for which_neuron in chosen_neuron
    example_map = place_map_all_1[:,:,which_neuron]
    mapped_map_all[:,:,which_neuron] = boundary_morphing_map(example_map, mask_pixels, mask_pixels_array_map, bin_interval)
end

corr_trials_all = fill(NaN32, size(place_map_all_1, 3))

for neuron_idx in chosen_neuron
    map_1 = place_map_all_2[:,:, neuron_idx]
    map_2 = mapped_map_all[:,:, neuron_idx]
    corr_trials_all[neuron_idx] = corr_2d(map_1, map_2)
end


list_offset = round.(Int32, (numpy.linspace(1, length(y1), 72)))

corr_trials_all_degree = fill(NaN32, length(list_offset), length(chosen_neuron))

@showprogress for (i,offset) in enumerate(list_offset)

    y1_roll = numpy.roll(y1, offset)
    y2_roll = numpy.roll(y2, offset);

    y_roll = [[y1_roll[i], y2_roll[i]] for i in 1:nr_dots];

    mask_pixels_array_map = map_dots_from_boundary.(mask_pixels_array, x= x, y =y_roll, k= 2);

    
    mapped_map_all = fill(NaN32, size(place_map_all_1, 1), size(place_map_all_1,2), length(chosen_neuron))
    for (i_neuron, which_neuron) in enumerate(chosen_neuron)
        example_map = place_map_all_1[:,:,which_neuron]
        mapped_map_all[:,:,i_neuron] = boundary_morphing_map(example_map, mask_pixels, mask_pixels_array_map, bin_interval)
    end


    corr_trials_all = fill(NaN32, length(chosen_neuron))

    for (i_neuron, which_neuron) in enumerate(chosen_neuron)
        map_1 = place_map_all_2[:,:, which_neuron]
        map_2 = mapped_map_all[:,:, i_neuron]
        corr_trials_all[i_neuron] = corr_2d(map_1, map_2)
    end
    
    
    corr_trials_all_degree[i, :] .= corr_trials_all
end




corr_trials_all_degree_mean = nanmean(corr_trials_all_degree, dims=2)[:,1]
best_offset = list_offset[findall(corr_trials_all_degree_mean .== nanmaximum(corr_trials_all_degree_mean))[1]]
offset = copy(best_offset)

y1_roll = numpy.roll(y1, offset)
y2_roll = numpy.roll(y2, offset);

y_roll = [[y1_roll[i], y2_roll[i]] for i in 1:nr_dots];

mask_pixels_array_map = map_dots_from_boundary.(mask_pixels_array, x= x, y =y_roll, k= 2);


mapped_map_all = fill(NaN32, size(place_map_all_1, 1), size(place_map_all_1,2), length(tel_place_cell_index))
for (i_neuron, which_neuron) in enumerate(tel_place_cell_index)
    example_map = place_map_all_1[:,:,which_neuron]
    mapped_map_all[:,:,i_neuron] = boundary_morphing_map(example_map, mask_pixels, mask_pixels_array_map, bin_interval)
end


corr_trials_all = fill(NaN32, length(tel_place_cell_index))

for (i_neuron, which_neuron) in enumerate(tel_place_cell_index)
    map_1 = place_map_all_2[:,:, which_neuron]
    map_2 = mapped_map_all[:,:, i_neuron]
    corr_trials_all[i_neuron] = corr_2d(map_1, map_2)
end
    

if precise_loc
    peak_loc_map_2_array = [peak_loc_map_2_precise[:,i] for i in 1:size(peak_loc_map_2, 2)]
else
    peak_loc_map_2_array = [(peak_loc_map_2[:,i].-0.5)*bin_interval for i in 1:size(peak_loc_map_2, 2)]
end

peak_loc_map_2_array_map = map_dots_from_boundary.(peak_loc_map_2_array, x= x, y =y_roll, k= 2);
    
    
    
    
h5open(joinpath(data_path(ds_save_analyzer_1), "boundary_morph_earlylate.h5"), "w") do file
    file["corr_trials_all"] = corr_trials_all
    file["mapped_map_all"] = mapped_map_all 
    file["corr_trials_all_degree"] = corr_trials_all_degree 
    file["chosen_neuron"] = chosen_neuron 
    file["tel_place_cell_index"] = tel_place_cell_index 
    file["peak_loc_map_2_array_map"] = collect(numpy.array(peak_loc_map_2_array_map)')
end;
    
end